In [2]:
import IPython
IPython.Application.instance().kernel.do_shutdown(True) # "True" is to restart


{'status': 'ok', 'restart': True}

In [1]:
pip install -r requirements.txt


[notice] A new release of pip available: 22.2.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install  accelerate bitsandbytes


[notice] A new release of pip available: 22.2.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [15]:
import logging
import torch
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from trl import SFTTrainer

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


In [16]:
available_gpus = [f'cuda:{i}' for i in range(torch.cuda.device_count())]
logger.info(f"Available CUDA devices: {available_gpus}")

INFO:__main__:Available CUDA devices: ['cuda:0', 'cuda:1', 'cuda:2', 'cuda:3']


In [17]:
model_name = "daryl149/Llama-2-7b-hf" 
dataset_name = "tatsu-lab/alpaca"


In [18]:
# Load dataset
data = load_dataset(dataset_name, split="train[0:10000]")
logger.info("Dataset loaded successfully.")

data = data.train_test_split(test_size=0.1)
train_dataset = data["train"]
test_dataset = data["test"]


INFO:__main__:Dataset loaded successfully.


In [32]:
for i in range(10):
    print(train_dataset[i])


{'instruction': 'Analyze the given poem and briefly explain its main message.', 'input': "The poem reads as follows:\n\nThe world will keep spinning\nAs it always has been\nBut that doesn't mean that you,\nAre ever really unseen", 'output': 'The poem conveys the message that everyone has unique value and is an irreplaceable part of the world, even though the world carries on as it always has. The poem reminds the reader not to take for granted the importance of their individual presence and contribution in life.', 'text': "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nAnalyze the given poem and briefly explain its main message.\n\n### Input:\nThe poem reads as follows:\n\nThe world will keep spinning\nAs it always has been\nBut that doesn't mean that you,\nAre ever really unseen\n\n### Response:\nThe poem conveys the message that everyone has unique value

In [35]:
from transformers import pipeline

# Initialize the pipeline for text generation with the model and tokenizer
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

def generate_model_output(instruction, input_text, generator):
    # Combine instruction and input text to form the prompt for the model
    prompt = f"{instruction} {input_text}"
    
    # Generate output using the model pipeline
    outputs = generator(prompt, max_length=50, num_return_sequences=1)
    
    # Extract the generated text and return it
    # Assuming that the model's output starts after the input text
    # You may need to adjust slicing depending on your model's behavior
    generated_text = outputs[0]["generated_text"][len(prompt):]
    
    return generated_text.strip()

# Example usage
for i in range(10):
    instruction = data["train"]["instruction"][i]
    input_text = data["train"]["input"][i]
    
    # Get the model's output
    output = generate_model_output(instruction, input_text, generator)
    print(f"Instruction: {instruction}")
    print(f"Input: {input_text}")
    print(f"Output: {output}")
    print("=" * 50)  # Separator for readability


The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausal

Instruction: Analyze the given poem and briefly explain its main message.
Input: The poem reads as follows:

The world will keep spinning
As it always has been
But that doesn't mean that you,
Are ever really unseen
Output: .
Instruction: Given a user query, suggest a relevant title.
Input: Query: Best hotels in London
Output: . Response: The Ritz London, The Savoy, The Langham, The Dorchester, The Connaught, The Savoy, The Goring,
Instruction: Add at least 2 more examples to this list: flight, eagle, hawk
Input: 
Output: Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Des
Instruction: Given the following list of numbers, identify the median.
Input: List: [3, 5, 19, 25, 32]
Output: 
Instruction: Oragnize the following statement into a proper syntax.
Input: Mike yesterday to go running but the drizzle rain stopped
Output: him.

### Input:
Mike yesterday to go running but the drizzle rain stopped him.
Instruction:

In [36]:
# Example instructions and inputs
example_data = [
    {
        "instruction": "Write a summary of the following text:",
        "input": "The history of AI dates back to antiquity. Ancient myths about artificial beings endowed with intelligence or consciousness have been common throughout history."
    },
    {
        "instruction": "Translate the following sentence to French:",
        "input": "Hello, how are you today?"
    },
    {
        "instruction": "Answer the question based on the passage provided:",
        "input": "Passage: 'The quick brown fox jumps over the lazy dog.' Question: 'Which animal is quick?'"
    }
]

# Call the generate_model_output function with the examples
for example in example_data:
    output = generate_model_output(example["instruction"], example["input"], generator)
    print(f"Instruction: {example['instruction']}")
    print(f"Input: {example['input']}")
    print(f"Output: {output}")
    print("=" * 50)  # Separator for readability


/opt/app-root/lib64/python3.9/site-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Instruction: Write a summary of the following text:
Input: The history of AI dates back to antiquity. Ancient myths about artificial beings endowed with intelligence or consciousness have been common throughout history.
Output: In the 19th century, the
Instruction: Translate the following sentence to French:
Input: Hello, how are you today?
Output: Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Create a list of 5
Instruction: Answer the question based on the passage provided:
Input: Passage: 'The quick brown fox jumps over the lazy dog.' Question: 'Which animal is quick?'
Output: Response: The quick brown fox jumps over the lazy dog. Below is


In [20]:
# Prepare tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
logger.info("Tokenizer prepared successfully.")


INFO:__main__:Tokenizer prepared successfully.


In [21]:

# Prepare model for quantization and load pretrained weights
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# current_device = 'cuda:0'
# torch.cuda.set_device(current_device)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map = 'auto',
    quantization_config=quantization_config,
)

model.resize_token_embeddings(len(tokenizer))
logger.info("Model loaded and token embeddings resized successfully.")


(…)9/llama-2-7b-hf/resolve/main/config.json: 100%|██████████| 578/578 [00:00<00:00, 158kB/s]
(…)esolve/main/pytorch_model.bin.index.json: 100%|██████████| 26.8k/26.8k [00:00<00:00, 14.5MB/s]
pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]
pytorch_model-00001-of-00002.bin:   0%|          | 10.5M/9.98G [00:00<02:28, 67.3MB/s]
pytorch_model-00001-of-00002.bin:   0%|          | 31.5M/9.98G [00:00<01:19, 126MB/s] 
pytorch_model-00001-of-00002.bin:   1%|          | 52.4M/9.98G [00:00<01:03, 156MB/s]
pytorch_model-00001-of-00002.bin:   1%|          | 73.4M/9.98G [00:00<00:58, 170MB/s]
pytorch_model-00001-of-00002.bin:   1%|          | 94.4M/9.98G [00:00<00:54, 181MB/s]
pytorch_model-00001-of-00002.bin:   1%|          | 115M/9.98G [00:00<00:52, 188MB/s] 
pytorch_model-00001-of-00002.bin:   1%|▏         | 136M/9.98G [00:00<00:50, 193MB/s]
pytorch_model-00001-of-00002.bin:   2%|▏         | 157M/9.98G [00:00<00:49, 196MB/s]
pytorch_model-00001-of-00002.bin:   2%|▏   

In [26]:

# Prepare model for k-bit training
model = prepare_model_for_kbit_training(model)

# Define PEFT configuration
peft_config = LoraConfig(
    lora_alpha= 8,
    lora_dropout= 0.1,
    r= 16,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj", "up_proj"]
)

model = get_peft_model(model, peft_config)
logger.info("PEFT configuration prepared successfully.")

# Define training arguments
use_fp16 = torch.cuda.is_available()
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    max_grad_norm=0.3,
    weight_decay= 0.001,
    optim="paged_adamw_8bit",
    max_steps=100,
    logging_steps=30,
    learning_rate=2e-4,
    fp16=use_fp16,
    warmup_ratio=0.1,
    lr_scheduler_type="linear",
    num_train_epochs=1,
    save_strategy="steps",
    save_total_limit=3,
    push_to_hub=False,
)
logger.info(f"Training arguments: {training_args}")

INFO:__main__:PEFT configuration prepared successfully.
INFO:__main__:Training arguments: TrainingArguments(
_n_gpu=4,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=True,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'fsdp_min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=2,
gradient_checkpointing=False,
greater_is_better=None,
group_by_leng

In [27]:

# Initialize the trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    dataset_text_field="text",
    max_seq_length=1024,
    tokenizer=tokenizer,
    args=training_args,
    packing=True,
    peft_config=peft_config,
)

logger.info("Training started.")
trainer.train()
logger.info("Training Complete.")


/opt/app-root/lib64/python3.9/site-packages/trl/trainer/sft_trainer.py:212: UserWarning: You passed `packing=True` to the SFTTrainer, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(
INFO:__main__:Training started.
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss
30,1.188900
60,0.912200
90,0.873300


/opt/app-root/lib64/python3.9/site-packages/trl/trainer/utils.py:268: UserWarning: The dataset reached end and the iterator is reset to the start.
  warnings.warn("The dataset reached end and the iterator is reset to the start.")
INFO:__main__:Training Complete.
INFO:__main__:Final model and tokenizer saved locally.


In [29]:
!nvidia-smi


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Fri Nov 10 17:44:43 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T

In [47]:
pip install adapter-transformers


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)

[notice] A new release of pip available: 22.2.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [94]:
# Save the model, configuration, and tokenizer to the specified directory
save_directory = "results"

# Save all necessary components for later use or inference
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)
model.config.save_pretrained(save_directory)

In [96]:
# Set the model to evaluation mode
model.eval()
logger.info("Model, configuration, and tokenizer saved to '%s'", save_directory)


INFO:__main__:Model, configuration, and tokenizer saved to 'results'


In [99]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

# Load your saved model and tokenizer
model_path = "results"
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Create a text generation pipeline
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Define the function for generating model output
def generate_model_output(instruction, input_text, generator):
    # Combine instruction and input text to form the prompt for the model
    prompt = f"{instruction} {input_text}"
    
    # Generate output using the model pipeline
    outputs = generator(prompt, max_length=50, num_return_sequences=1)
    
    # Extract the generated text and return it
    generated_text = outputs[0]['generated_text']
    
    # Assuming the model's generated output is at the end after input text
    return generated_text[len(prompt):].strip()

# Example instructions and inputs
example_data = [
    {
        "instruction": "Write a summary of the following text:",
        "input": "The history of AI dates back to antiquity. Ancient myths about artificial beings endowed with intelligence or consciousness have been common throughout history."
    },
    {
        "instruction": "Translate the following sentence to French:",
        "input": "Hello, how are you today?"
    },
    {
        "instruction": "Answer the question based on the passage provided:",
        "input": "Passage: 'The quick brown fox jumps over the lazy dog.' Question: 'Which animal is quick?'"
    }
]

# Example usage
for example in example_data:
    instruction = example["instruction"]
    input_text = example["input"]
    
    # Get the model's output
    output = generate_model_output(instruction, input_text, generator)
    print(f"Instruction: {instruction}")
    print(f"Input: {input_text}")
    print(f"Output: {output}")
    print("=" * 50)  # Separator for readability


Loading checkpoint shards: 100%|██████████| 3/3 [00:12<00:00,  4.23s/it]
Some weights of LlamaForCausalLM were not initialized from the model checkpoint at results and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Instruction: Write a summary of the following text:
Input: The history of AI dates back to antiquity. Ancient myths about artificial beings endowed with intelligence or consciousness have been common throughout history.
Output: Tra主oder jú vuelnederbördITE ORDER Див
Instruction: Translate the following sentence to French:
Input: Hello, how are you today?
Output: rolledindaSError referring removes음method vertex Sainтейikan Englandchromik britannique production require Japanese doiцима oùцимаaux expectedocrat Roth claim trabaj requestsieg profileserkрес Emb bu
Instruction: Answer the question based on the passage provided:
Input: Passage: 'The quick brown fox jumps over the lazy dog.' Question: 'Which animal is quick?'
Output: Dist Federal� améric tec Cultural"])нин Sau со Gesch
